# Contextual Forests V2

### 1. BabelNet Iinterface

In [24]:
import requests
import json
import wikipediaapi

#GLOBAL VARIABLES
wiki = wikipediaapi.Wikipedia('en')
KEY  = 'b361cac5-2b53-40a8-8df8-6a2cfb140491' #Register in BabelNet to get one
BASE = 'https://babelnet.io/v5'
API = {
    "get_info": "getSynset",
    "get_synsets":"getSynsetIds",
    "get_relations":"getOutgoingEdges"
}

def __format_arguments(args):
    """
    Desc: Returns a formatted string for making an HTML request
    Params: - args (dict): dictionary for params values, format is {paramName:paramValue ...}
    Returns: (String) formatted string
    """
    return "&".join(["{}={}".format(key,value) for key,value in args.items()])

def __make_request(args,op,verbose=False):
    """
    Desc: Makes arequest to BabelNet API given a parametters dictionary
    Params: - args (dict): dictionary for params values, format is {paramName:paramValue ...}
            - op (string): operation, choose between {get_info,get_synsets,get_relations}
            - verbose (bool): show messages 
    Returns: (Dict) BabelNet's API response
    """
    args["key"] = KEY
    args["targetLang"] = "EN"
    r = requests.get("{}/{}?{}".format(BASE,API[op],__format_arguments(args)))
    if verbose:
        print("Status code: {}".format(r.status_code))
    return r.json() if r.ok else None

def synset_info(synset_id,verbose=False):
    """
    Desc: Returns the information associated with a BabelNet synset
    Params: - synset_id (String): BabelNet synset id
            - verbose (bool): show messages 
    Returns: (Dict) BabelNet's sysnset information
    """
    return __make_request({"id":synset_id},"get_info",verbose)

def synset_relations(synset_id,verbose=False):
    """
    Desc: Returns the outgoing semantic relations of a synset
    Params: - synset_id (String): BabelNet synset id
            - verbose (bool): show messages 
    Returns: (Dict) BabelNet's sysnset outgoing conexions
    """
    return __make_request({"id":synset_id},"get_relations",verbose)
   
def general_categories(synset_id):
    """  
    Desc: Returns the ids of the general categories a synset belongs to
    Params: - synset_id (String): BabelNet synset id
    Returns: (List)[String] ids 
    """
    categories = [relation["target"] for relation in synset_relations(synset_id) 
                  if relation["pointer"]["shortName"] in ["is-a","part_of","subclass_of"]]
    res = []
    for cat in categories:
        try:
            res.append(synset_info(cat)["senses"][0]["properties"]["synsetID"]["id"] )
        except IndexError:
            continue
    return res

def wiki_page(synset_id):
    """  
    Desc: Return the wikipedia page associated with a synset id
    Params: - synset_id (String): BabelNet synset id
    Returns: (Wikipediaapi.Page) wikipedia page object
    """
    synset = synset_info(synset_id)
    for sense in synset["senses"]:
        if sense['properties']["source"] == "WIKI":
            return wiki.page(sense['properties']["fullLemma"].replace("_"," "))
        else:
            continue
    return None
        

def search_synsets(lemma,pos,searchLang="EN",verbose=False):
    """  
    Desc: Returns the ids of possible synsets associated to the provided word
    Params: - lemma (String): word for searching
            - pos (String): position (NOUN,VERB ...)
            - searchLang (String): language for search and results
            - verbose (bool): show messages 
    Returns: (List)[String]: ids 
    """
    args = {
        "lemma":lemma,
        "pos":pos,
        "searchLang":searchLang,
        "key":KEY
    }
    return [ele["id"] for ele in __make_request(args,"get_synsets",verbose)]


#search_synsets("apple","NOUN")   
#general_categories("bn:00289737n")
#wiki_page("bn:00289737n").text

        
    
                   

'Apple were a British psychedelic rock band. The band was founded in Cardiff in 1968 by Rob Ingram on guitar and Jeff Harrad on bass. They released a single LP in 1969, titled An Apple a Day. The album was a commercial failure, and the band ceased to exist shortly after its release. However, during the subsequent years several tracks from the LP, most notably "The Otherside" by Harrad, were dubbed classics of British psychedelic rock by critics, making An Apple a Day one of the most sought-after British psychedelic rarities.\nThe original vinyl version (released by Page One Records) is now extremely hard to find. A reissue by Repertoire Records was released in 1994, which included several bonus songs (those being early mono versions of some of the album\'s tracks).\n\nMembers\nDennis Regan, vocals\nRobbo Ingram, guitar\nJeff Harrad, bass\nCharlie Barber, piano\nDave Brassington, drums\n\nDiscography\nAlbums\nAn Apple a Day (Page One POLS 016) 1969\n\nSingles\n"Thank You Very Much" / "Y

### From terms to targetted texts